In [2]:
#importando as bibliotecas que serão utilizadas
import duckdb
import pandas as pd
import os
from datetime import datetime

In [3]:
#criando conexão 
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [19]:
#leitura dos arquivos, incluindo nome do arquivo, data e hora da ingestão
arquivo = 'z0019_2.csv'
data_ingestao = datetime.now()
df = pd.read_csv(f'../landing/{arquivo}', sep=';')
df['NM_ARQUIVO'] = arquivo
df['DT_INGESTAO'] = data_ingestao
df.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,NM_ARQUIVO,DT_INGESTAO
0,10004,SERRA,BT50,100,200,z0019_2.csv,2026-02-22 12:00:06.666797
1,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-02-22 12:00:06.666797
2,10003,PREGO,BT10,100,60,z0019_2.csv,2026-02-22 12:00:06.666797


In [20]:
#criação de uma tabela da camada bronze
con.execute("""
    CREATE TABLE IF NOT EXISTS bronze_produtos(
            NATBR VARCHAR,
            MAKTX VARCHAR,
            WERKS VARCHAR,
            MAINS VARCHAR,
            LABST VARCHAR,
            NM_ARQUIVO VARCHAR,
            DT_INGESTAO TIMESTAMP,
            )
""")

In [21]:
#ingestão dos dados na tabela da camada bronze
con.execute("""INSERT INTO bronze_produtos SELECT * FROM df""")

In [25]:
#consulta na bronze -> volta no passo de leitura do dataframe e troca para arquivo 2
resultado = con.execute("SELECT * FROM bronze_z0019").fetchdf()
resultado.head(10)


,NATBR,MAKTX,WERKS,MAINS,LABST,NM_ARQUIVO,DT_INGESTAO
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-22 11:35:22.232150
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-22 11:35:22.232150
2,10003,PREGO,BT10,100,50,z0019_1.csv,2026-02-22 11:35:22.232150
3,10004,SERRA,BT50,100,200,z0019_2.csv,2026-02-22 12:00:06.666797
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-02-22 12:00:06.666797
5,10003,PREGO,BT10,100,60,z0019_2.csv,2026-02-22 12:00:06.666797


In [24]:
#alterar o nome da tabela bronze para seu nome_bruto
con.execute("ALTER TABLE bronze_produtos RENAME TO bronze_z0019")

In [1]:
#encerrar conexão com o banco de dados
con.close()

NameError: name 'con' is not defined